#Using a Transformer

In [72]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, Input, Concatenate, RepeatVector, Lambda, LSTM, Dot, Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertModel

In [73]:

df = pd.read_csv("/content/drive/MyDrive/test.csv")
df.head()
df.shape
df_train = df[:100]
df_train.shape
df_test = df[100:110]
df_train.head()
df_test.shape
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def clean_text(review):
    review = re.sub('[^a-zA-Z0-1]', ' ', review)
    review = review.lower()
    review = review.split()


    review = ' '.join(review)
    return review

In [74]:
article_train = df_train['article'].values.tolist()
highlights_train = df_train['highlights'].values.tolist()
article_test = df_test['article'].values.tolist()
highlights_test = df_test['highlights'].values.tolist()

In [75]:
!pip install tensorflow tensorflow-addons transformers

In [76]:
pip install --upgrade tensorflow-addons


In [77]:
!pip install transformers


In [78]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM



#Using TFIDF vectorizer

In [35]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.translate.bleu_score import sentence_bleu

# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
# Tokenize articles into sentences
sentences_train = [word_tokenize(sent.lower()) for article in article_train for sent in sent_tokenize(article)]
sentences_test = [word_tokenize(sent.lower()) for article in article_test for sent in sent_tokenize(article)]

# Flatten the list of sentences for TF-IDF vectorization
flat_sentences_train = [' '.join(sent) for sent in sentences_train]
flat_sentences_test = [' '.join(sent) for sent in sentences_test]

# Create TF-IDF vectors after removing stop words
stop_words = set(stopwords.words('english'))

vectorizer = TfidfVectorizer(stop_words='english')  # Set stop_words to 'english'

In [41]:
tfidf_matrix_train = vectorizer.fit_transform(flat_sentences_train)
tfidf_matrix_test = vectorizer.transform(flat_sentences_test)

In [42]:
def generate_summary(article, num_words=50):
    article_tfidf = vectorizer.transform([article.lower()])
    similarity_matrix = cosine_similarity(article_tfidf, tfidf_matrix_train)

    # Get the indices of the most important sentences
    top_sentences_indices = similarity_matrix.argsort()[0][-num_words:][::-1]

    # Sort the sentences based on their original order in the document
    top_sentences_indices = sorted(top_sentences_indices)

    # Extract the top sentences from the original article
    top_sentences = [flat_sentences_train[i] for i in top_sentences_indices]

    # Combine the top sentences to form the summary
    summary = ' '.join(top_sentences)

    return summary

In [43]:
predicted_summaries = [generate_summary(article) for article in article_test]

# Evaluate using BLEU metric
bleu_scores = [sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(highlights_test, predicted_summaries) if gen]
mean_bleu_score = sum(bleu_scores) / len(bleu_scores)

print("Mean BLEU Score:", mean_bleu_score)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Mean BLEU Score: 0.0003756286840992494


#Using Encoder Decoder Sequence

In [53]:
# Choose a larger T5 model
model_name_t5 = "t5-base"

# Or an even larger one
model_name_t5 = "t5-large"

# For the largest T5 model
model_name_t5 = "t5-3.5-turbo"


In [79]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [81]:
def preprocess_sequences(sequences, tokenizer, max_length):
    # Tokenize sequences and convert to numerical IDs
    tokenized_sequences = [tokenizer(seq, truncation=True, padding='max_length')['input_ids'] for seq in sequences]

    # No need to pad_sequences for BERT-based models
    return tokenized_sequences


In [82]:
tokenized_articles_train = preprocess_sequences(article_train, tokenizer, max_length_article)
tokenized_summaries_train = preprocess_sequences(highlights_train, tokenizer, max_length_summary)
tokenized_articles_test = preprocess_sequences(article_test, tokenizer, max_length_article)
tokenized_summaries_test = preprocess_sequences(highlights_test, tokenizer, max_length_summary)



In [83]:
# Assuming you have tokenized sequences for both articles and summaries
input_ids_train = tokenizer_transformer(article_train, return_tensors="tf", padding=True, truncation=True, max_length=max_length_article)
decoder_input_ids_train = tokenizer_transformer(highlights_train, return_tensors="tf", padding=True, truncation=True, max_length=max_length_summary)





In [84]:
model_transformer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_transformer.fit(
    {'input_ids': input_ids_train['input_ids'], 'attention_mask': input_ids_train['attention_mask'], 'decoder_input_ids': decoder_input_ids_train['input_ids']},
    decoder_input_ids_train['input_ids'],
    epochs=50, batch_size=16, validation_split=0.2
)

Epoch 1/50
5/5 [==============================] - 72s 2s/step - loss: 10.3782 - accuracy: 0.0122 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 2/50
5/5 [==============================] - 5s 1s/step - loss: 10.3775 - accuracy: 0.0141 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 3/50
5/5 [==============================] - 5s 1s/step - loss: 10.3775 - accuracy: 0.0146 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 4/50
5/5 [==============================] - 5s 981ms/step - loss: 10.3775 - accuracy: 0.0153 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 5/50
5/5 [==============================] - 5s 965ms/step - loss: 10.3775 - accuracy: 0.0136 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 6/50
5/5 [==============================] - 5s 1s/step - loss: 10.3775 - accuracy: 0.0146 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 7/50
5/5 [==============================] - 5s 964ms/step - loss: 10.3775 - accuracy: 0.0150 - val_loss: 10.3775 - val_accuracy: 0.0135
Epoch 8/50
5/5 [=

In [85]:
input_ids_test = tokenizer_transformer(article_test, return_tensors="tf", padding=True, truncation=True, max_length=max_length_article)


In [86]:
predicted_summaries = []
for input_seq in input_ids_test['input_ids']:
    # Use the trained model to generate summaries
    input_tensor = tf.constant(input_seq[tf.newaxis, :])
    attention_mask = tf.ones_like(input_tensor)
    predicted_ids = model_transformer.generate(input_tensor, attention_mask=attention_mask, max_length=max_length_summary, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Decode the generated IDs to text
    predicted_summary = tokenizer_transformer.decode(predicted_ids[0], skip_special_tokens=True)

    predicted_summaries.append(predicted_summary)

# Now 'predicted_summaries' contains the generated summaries for your test articles


In [87]:
len(predicted_summaries)

10